In [1]:
import pandas as pd
import numpy as np
import nltk
import pyarabic.arabrepr
from tashaphyne.stemming import ArabicLightStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

In [2]:
def file(file_pass):
    df = pd.read_csv(file_pass)
    return df
file('processed_data.csv')
df = file('processed_data.csv')
df

,Title,Content,Link,Category
0,تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...,جاءت جوله التمويل التمهيديه الاولي للشركه بقيا...,https://ryadiybusiness.com/%d8%aa%d9%8a%d8%b1%...,ريادة أعمال
1,تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه,تعد دراسه الجدوي متطلبا اساسيا لنجاح اي مشروع ...,https://ryadiybusiness.com/%d9%83%d9%8a%d9%81-...,ريادة أعمال
2,منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...,وقالت لجنه نوبل للمره الاولي التاريخ يمكن الان...,https://arabic.sputniknews.com/science/2020100...,علوم وتكنولوجيا
3,هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...,نشرت الوكاله وثاءق علي الانترنت اعدها موظفوها ...,https://arabic.sputniknews.com/world/202012151...,علوم وتكنولوجيا
4,حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...,دراسه حديثه قام فريق العلماء جونز هوبكنز ميديس...,https://arabic.rt.com//technology/1161864-%D8%...,علوم وتكنولوجيا
...,...,...,...,...
8394,قواعد اوروبيه تهدد بخفض 10 ايرادات شركات التكن...,قالت وكاله بلومبرغ الاثنين ان كبري شركات التكن...,https://asharq.com/ar/7vNsbHVkd9fTQhBQI70ASx-ق...,علوم وتكنولوجيا
8395,حوار خاص مهندس سعودي نباتك فكره تهدف لزراعه مل...,بدات الفكره نعومه اظفاره عندما كان السابعه عمر...,https://news.un.org/ar/story/2020/09/1062802,أخرى
8396,كريم تطلق خدمه اطلب اي شيء الجديده,اعلنت منصه كريم 4 فبراير الجاري اطلاق خدمه اطل...,https://ryadiybusiness.com/%d9%83%d8%b1%d9%8a%...,ريادة أعمال
8397,تقرير اممي جديد يشير الي تاثير جاءحه فيروس كور...,وفقا لتقديرات المكتب الاممي المعني بالمخدرات و...,https://news.un.org/ar/story/2020/06/1057232,أخرى


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8399 entries, 0 to 8398
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     8399 non-null   object
 1   Content   8386 non-null   object
 2   Link      8399 non-null   object
 3   Category  8399 non-null   object
dtypes: object(4)
memory usage: 262.6+ KB


In [4]:
df.isna().count()

Title       8399
Content     8399
Link        8399
Category    8399
dtype: int64

In [5]:
df['Category'].value_counts()

أخرى               2800
علوم وتكنولوجيا    2800
ريادة أعمال        2799
Name: Category, dtype: int64

In [6]:
llist=[]
for z in df['Content']:
    z = str(z)
    llist.append(z)
df['Content']=llist

# segmentation

In [7]:
def segmentation(df):
    sent_token=nltk.data.load('tokenizers/punkt/english.pickle')
    for qi in df['Content']:
        df['Content'][qi]=sent_token.tokenize(qi)
    return df
df = segmentation(df)
df

,Title,Content,Link,Category
0,تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...,جاءت جوله التمويل التمهيديه الاولي للشركه بقيا...,https://ryadiybusiness.com/%d8%aa%d9%8a%d8%b1%...,ريادة أعمال
1,تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه,تعد دراسه الجدوي متطلبا اساسيا لنجاح اي مشروع ...,https://ryadiybusiness.com/%d9%83%d9%8a%d9%81-...,ريادة أعمال
2,منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...,وقالت لجنه نوبل للمره الاولي التاريخ يمكن الان...,https://arabic.sputniknews.com/science/2020100...,علوم وتكنولوجيا
3,هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...,نشرت الوكاله وثاءق علي الانترنت اعدها موظفوها ...,https://arabic.sputniknews.com/world/202012151...,علوم وتكنولوجيا
4,حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...,دراسه حديثه قام فريق العلماء جونز هوبكنز ميديس...,https://arabic.rt.com//technology/1161864-%D8%...,علوم وتكنولوجيا
...,...,...,...,...
8394,قواعد اوروبيه تهدد بخفض 10 ايرادات شركات التكن...,قالت وكاله بلومبرغ الاثنين ان كبري شركات التكن...,https://asharq.com/ar/7vNsbHVkd9fTQhBQI70ASx-ق...,علوم وتكنولوجيا
8395,حوار خاص مهندس سعودي نباتك فكره تهدف لزراعه مل...,بدات الفكره نعومه اظفاره عندما كان السابعه عمر...,https://news.un.org/ar/story/2020/09/1062802,أخرى
8396,كريم تطلق خدمه اطلب اي شيء الجديده,اعلنت منصه كريم 4 فبراير الجاري اطلاق خدمه اطل...,https://ryadiybusiness.com/%d9%83%d8%b1%d9%8a%...,ريادة أعمال
8397,تقرير اممي جديد يشير الي تاثير جاءحه فيروس كور...,وفقا لتقديرات المكتب الاممي المعني بالمخدرات و...,https://news.un.org/ar/story/2020/06/1057232,أخرى


# Tokenization

In [8]:
def Tokenization(df):
    df['Content'] = df['Content'].apply(nltk.word_tokenize)
    return df
df = Tokenization(df)
df

,Title,Content,Link,Category
0,تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...,"[جاءت, جوله, التمويل, التمهيديه, الاولي, للشرك...",https://ryadiybusiness.com/%d8%aa%d9%8a%d8%b1%...,ريادة أعمال
1,تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه,"[تعد, دراسه, الجدوي, متطلبا, اساسيا, لنجاح, اي...",https://ryadiybusiness.com/%d9%83%d9%8a%d9%81-...,ريادة أعمال
2,منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...,"[وقالت, لجنه, نوبل, للمره, الاولي, التاريخ, يم...",https://arabic.sputniknews.com/science/2020100...,علوم وتكنولوجيا
3,هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...,"[نشرت, الوكاله, وثاءق, علي, الانترنت, اعدها, م...",https://arabic.sputniknews.com/world/202012151...,علوم وتكنولوجيا
4,حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...,"[دراسه, حديثه, قام, فريق, العلماء, جونز, هوبكن...",https://arabic.rt.com//technology/1161864-%D8%...,علوم وتكنولوجيا
...,...,...,...,...
8394,قواعد اوروبيه تهدد بخفض 10 ايرادات شركات التكن...,"[قالت, وكاله, بلومبرغ, الاثنين, ان, كبري, شركا...",https://asharq.com/ar/7vNsbHVkd9fTQhBQI70ASx-ق...,علوم وتكنولوجيا
8395,حوار خاص مهندس سعودي نباتك فكره تهدف لزراعه مل...,"[بدات, الفكره, نعومه, اظفاره, عندما, كان, السا...",https://news.un.org/ar/story/2020/09/1062802,أخرى
8396,كريم تطلق خدمه اطلب اي شيء الجديده,"[اعلنت, منصه, كريم, 4, فبراير, الجاري, اطلاق, ...",https://ryadiybusiness.com/%d9%83%d8%b1%d9%8a%...,ريادة أعمال
8397,تقرير اممي جديد يشير الي تاثير جاءحه فيروس كور...,"[وفقا, لتقديرات, المكتب, الاممي, المعني, بالمخ...",https://news.un.org/ar/story/2020/06/1057232,أخرى


# stopwords Removing

In [9]:
stopw = pd.read_csv('arabic-stop-words.csv')
def stopwords_Removing(df):
    gnewl=[]
    for i0 in df['Content']:
        newlist1=[]
        for word0 in i0:
            if word0 not in stopw:
                newlist1.append(word0)
        gnewl.append(newlist1)
    df['Content']=gnewl
    return df
df = stopwords_Removing(df)
df

,Title,Content,Link,Category
0,تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...,"[جاءت, جوله, التمويل, التمهيديه, الاولي, للشرك...",https://ryadiybusiness.com/%d8%aa%d9%8a%d8%b1%...,ريادة أعمال
1,تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه,"[تعد, دراسه, الجدوي, متطلبا, اساسيا, لنجاح, اي...",https://ryadiybusiness.com/%d9%83%d9%8a%d9%81-...,ريادة أعمال
2,منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...,"[وقالت, لجنه, نوبل, للمره, الاولي, التاريخ, يم...",https://arabic.sputniknews.com/science/2020100...,علوم وتكنولوجيا
3,هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...,"[نشرت, الوكاله, وثاءق, علي, الانترنت, اعدها, م...",https://arabic.sputniknews.com/world/202012151...,علوم وتكنولوجيا
4,حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...,"[دراسه, حديثه, قام, فريق, العلماء, جونز, هوبكن...",https://arabic.rt.com//technology/1161864-%D8%...,علوم وتكنولوجيا
...,...,...,...,...
8394,قواعد اوروبيه تهدد بخفض 10 ايرادات شركات التكن...,"[قالت, وكاله, بلومبرغ, الاثنين, ان, كبري, شركا...",https://asharq.com/ar/7vNsbHVkd9fTQhBQI70ASx-ق...,علوم وتكنولوجيا
8395,حوار خاص مهندس سعودي نباتك فكره تهدف لزراعه مل...,"[بدات, الفكره, نعومه, اظفاره, عندما, كان, السا...",https://news.un.org/ar/story/2020/09/1062802,أخرى
8396,كريم تطلق خدمه اطلب اي شيء الجديده,"[اعلنت, منصه, كريم, 4, فبراير, الجاري, اطلاق, ...",https://ryadiybusiness.com/%d9%83%d8%b1%d9%8a%...,ريادة أعمال
8397,تقرير اممي جديد يشير الي تاثير جاءحه فيروس كور...,"[وفقا, لتقديرات, المكتب, الاممي, المعني, بالمخ...",https://news.un.org/ar/story/2020/06/1057232,أخرى


# Stemming

light based stemmer

In [10]:
ArListem = ArabicLightStemmer()
# def light_based_stemmer(df):
#     df['Content'] = df['Content'].apply(lambda x: [ArListem.light_stem(y) for y in x])
#     return df
# df = light_based_stemmer(df)    
# df

root-base stemmer

In [11]:
def root_base_stemmer(df):
    nnewl=[]
    for i in df['Content']:
        newl=[]
        for word in i:
            stem = ArListem.light_stem(str(word))
            l =ArListem.get_root()
            newl.append(l)
        nnewl.append(newl)
    df['Content']=nnewl
    return df
df = root_base_stemmer(df)    
df

,Title,Content,Link,Category
0,تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...,"[جوء, جول, مول, مهد, ولي, شرر, قود, شرر, Lever...",https://ryadiybusiness.com/%d8%aa%d9%8a%d8%b1%...,ريادة أعمال
1,تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه,"[عدد, درس, جدو, طلب, وسس, جوح, , شرع, نشء, بغض...",https://ryadiybusiness.com/%d9%83%d9%8a%d9%81-...,ريادة أعمال
2,منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...,"[قلت, لجن, نبل, مره, ولي, ورخ, مكن, ولن, شفف, ...",https://arabic.sputniknews.com/science/2020100...,علوم وتكنولوجيا
3,هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...,"[شرر, وكل, ثءق, على, نرن, وعد, وظف, عون, شرر, ...",https://arabic.sputniknews.com/world/202012151...,علوم وتكنولوجيا
4,حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...,"[درس, حدث, قام, ريق, علم, جنز, هبكنز, مدس, حدد...",https://arabic.rt.com//technology/1161864-%D8%...,علوم وتكنولوجيا
...,...,...,...,...
8394,قواعد اوروبيه تهدد بخفض 10 ايرادات شركات التكن...,"[قلت, وله, لمبرغ, ثنن, ن, كبر, شرك, كنلج, مرك,...",https://asharq.com/ar/7vNsbHVkd9fTQhBQI70ASx-ق...,علوم وتكنولوجيا
8395,حوار خاص مهندس سعودي نباتك فكره تهدف لزراعه مل...,"[بدو, فكر, عوم, ظفر, عندم, كان, سبع, عمر, زرع,...",https://news.un.org/ar/story/2020/09/1062802,أخرى
8396,كريم تطلق خدمه اطلب اي شيء الجديده,"[علن, نصص, ريم, 4, برر, جرو, طلق, خدم, طلب, , ...",https://ryadiybusiness.com/%d9%83%d8%b1%d9%8a%...,ريادة أعمال
8397,تقرير اممي جديد يشير الي تاثير جاءحه فيروس كور...,"[وفق, قدر, كتب, لمم, لمع, خدر, جرم, عوم, 2020,...",https://news.un.org/ar/story/2020/06/1057232,أخرى


In [12]:
df['Content'] = df['Content'].apply(lambda x: ' '.join(x))
df

,Title,Content,Link,Category
0,تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...,جوء جول مول مهد ولي شرر قود شرر Lever VCC شرك ...,https://ryadiybusiness.com/%d8%aa%d9%8a%d8%b1%...,ريادة أعمال
1,تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه,عدد درس جدو طلب وسس جوح شرع نشء بغض نظر حجم و...,https://ryadiybusiness.com/%d9%83%d9%8a%d9%81-...,ريادة أعمال
2,منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...,قلت لجن نبل مره ولي ورخ مكن ولن شفف فرس هوب كب...,https://arabic.sputniknews.com/science/2020100...,علوم وتكنولوجيا
3,هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...,شرر وكل ثءق على نرن وعد وظف عون شرر حلل سلم فع...,https://arabic.sputniknews.com/world/202012151...,علوم وتكنولوجيا
4,حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...,درس حدث قام ريق علم جنز هبكنز مدس حدد علم وحد ...,https://arabic.rt.com//technology/1161864-%D8%...,علوم وتكنولوجيا
...,...,...,...,...
8394,قواعد اوروبيه تهدد بخفض 10 ايرادات شركات التكن...,قلت وله لمبرغ ثنن ن كبر شرك كنلج مرك عملق وجه ...,https://asharq.com/ar/7vNsbHVkd9fTQhBQI70ASx-ق...,علوم وتكنولوجيا
8395,حوار خاص مهندس سعودي نباتك فكره تهدف لزراعه مل...,بدو فكر عوم ظفر عندم كان سبع عمر زرع حمد خلد و...,https://news.un.org/ar/story/2020/09/1062802,أخرى
8396,كريم تطلق خدمه اطلب اي شيء الجديده,علن نصص ريم 4 برر جرو طلق خدم طلب شيء جدد طلب...,https://ryadiybusiness.com/%d9%83%d8%b1%d9%8a%...,ريادة أعمال
8397,تقرير اممي جديد يشير الي تاثير جاءحه فيروس كور...,وفق قدر كتب لمم لمع خدر جرم عوم 2020 فون حوالى...,https://news.un.org/ar/story/2020/06/1057232,أخرى


In [13]:
# leCategory=LabelEncoder()
# df['Category'] = leCategory.fit_transform(df['Category'])
# df

In [14]:
Category_lebels = {"أخرى":3 , "علوم وتكنولوجيا":2, "ريادة أعمال":1}
df = df.replace({"Category": Category_lebels})
df

,Title,Content,Link,Category
0,تيرتل تري لابز للتقنيات الحيويه تحصل علي تمويل...,جوء جول مول مهد ولي شرر قود شرر Lever VCC شرك ...,https://ryadiybusiness.com/%d8%aa%d9%8a%d8%b1%...,1
1,تعد دراسه الجدوي لمشروعك الناشء 6 عوامل اساسيه,عدد درس جدو طلب وسس جوح شرع نشء بغض نظر حجم و...,https://ryadiybusiness.com/%d9%83%d9%8a%d9%81-...,1
2,منح جاءزه نوبل الطب لاكتشاف مجال التهاب الكبد ...,قلت لجن نبل مره ولي ورخ مكن ولن شفف فرس هوب كب...,https://arabic.sputniknews.com/science/2020100...,2
3,هيءه الغذاء والدواء الامريكيه لقاح مودرنا حقق ...,شرر وكل ثءق على نرن وعد وظف عون شرر حلل سلم فع...,https://arabic.sputniknews.com/world/202012151...,2
4,حظر بروتين الجهاز المناعي يساعد علي محاربه عدو...,درس حدث قام ريق علم جنز هبكنز مدس حدد علم وحد ...,https://arabic.rt.com//technology/1161864-%D8%...,2
...,...,...,...,...
8394,قواعد اوروبيه تهدد بخفض 10 ايرادات شركات التكن...,قلت وله لمبرغ ثنن ن كبر شرك كنلج مرك عملق وجه ...,https://asharq.com/ar/7vNsbHVkd9fTQhBQI70ASx-ق...,2
8395,حوار خاص مهندس سعودي نباتك فكره تهدف لزراعه مل...,بدو فكر عوم ظفر عندم كان سبع عمر زرع حمد خلد و...,https://news.un.org/ar/story/2020/09/1062802,3
8396,كريم تطلق خدمه اطلب اي شيء الجديده,علن نصص ريم 4 برر جرو طلق خدم طلب شيء جدد طلب...,https://ryadiybusiness.com/%d9%83%d8%b1%d9%8a%...,1
8397,تقرير اممي جديد يشير الي تاثير جاءحه فيروس كور...,وفق قدر كتب لمم لمع خدر جرم عوم 2020 فون حوالى...,https://news.un.org/ar/story/2020/06/1057232,3


tf-idf

In [15]:
vec = TfidfVectorizer()
features = vec.fit_transform(df['Content'])
pd.DataFrame(features.toarray(),columns=vec.vocabulary_.keys())

,جوء,جول,مول,مهد,ولي,شرر,قود,lever,vcc,شرك,...,2009قلت,لتقلمغده,لتقلمبم,قررل,لتقررعل,سقستند,لصلهغده,علختتمت,ردتهمقد,لمعرضهضح
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
x_train,x_test,y_train,y_test = train_test_split(features,df['Category'] ,test_size=0.2)

In [17]:
model = MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB()

In [18]:
pred = model.predict(x_test)

In [19]:
accuracy_score(y_test, pred)

0.9285714285714286

In [20]:
print(confusion_matrix(y_test, pred))

[[520   6  15]
 [ 34 470  63]
 [  2   0 570]]


counting

In [21]:
vec2 = CountVectorizer(binary=False)
features2 = vec2.fit_transform(df['Content'])

In [22]:
pd.DataFrame(features2.toarray(),columns=vec.vocabulary_.keys())

,جوء,جول,مول,مهد,ولي,شرر,قود,lever,vcc,شرك,...,2009قلت,لتقلمغده,لتقلمبم,قررل,لتقررعل,سقستند,لصلهغده,علختتمت,ردتهمقد,لمعرضهضح
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
x2_train,x2_test,y2_train,y2_test = train_test_split(features2,df['Category'] ,test_size=0.2)
model2 = MultinomialNB()
model2.fit(x2_train,y2_train)

MultinomialNB()

In [24]:
pred2 = model2.predict(x2_test)
accuracy_score(y2_test, pred2)

0.9660714285714286

In [25]:
print(confusion_matrix(y2_test, pred2))

[[542   7   4]
 [ 19 534  13]
 [  7   7 547]]


binary encoding

In [26]:
vec = CountVectorizer(binary=True)
features3 = vec.fit_transform(df['Content'])

In [27]:
pd.DataFrame(features3.toarray(),columns=vec.vocabulary_.keys())

,جوء,جول,مول,مهد,ولي,شرر,قود,lever,vcc,شرك,...,2009قلت,لتقلمغده,لتقلمبم,قررل,لتقررعل,سقستند,لصلهغده,علختتمت,ردتهمقد,لمعرضهضح
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
x3_train,x3_test,y3_train,y3_test = train_test_split(features3,df['Category'] ,test_size=0.2)
model3 = MultinomialNB()
model3.fit(x3_train,y3_train)

MultinomialNB()

In [29]:
pred3 = model3.predict(x3_test)
accuracy_score(y3_test, pred3)

0.9583333333333334

In [30]:
print(confusion_matrix(y3_test, pred3))

[[556   5  12]
 [ 28 499  19]
 [  3   3 555]]
